# Download and unpack data

In [1]:
!mkdir data
!wget https://s3.amazonaws.com/code2vec/data/java-small_data.tar.gz
!tar -xvzf java-small_data.tar.gz
!mv java-small data

--2020-06-02 16:23:14--  https://s3.amazonaws.com/code2vec/data/java-small_data.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.93.229
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.93.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 382731969 (365M) [application/x-tar]
Saving to: ‘java-small_data.tar.gz’

java-small_data.tar 100%[===================>] 365.00M  31.7MB/s    in 13s     

2020-06-02 16:23:27 (28.9 MB/s) - ‘java-small_data.tar.gz’ saved [382731969/382731969]

java-small/
java-small/java-small.dict.c2v
java-small/java-small.train.c2v
java-small/java-small.val.c2v
java-small/java-small.test.c2v


# Load from the github

In [2]:
!git clone https://github.com/Vitaly-Protasov/DL_project_skoltech

Cloning into 'DL_project_skoltech'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 135 (delta 75), reused 88 (delta 35), pack-reused 0
Receiving objects: 100% (135/135), 28.55 KiB | 811.00 KiB/s, done.
Resolving deltas: 100% (75/75), done.


In [3]:
cd DL_project_skoltech/

/content/DL_project_skoltech


# Import our modules and torch


In [0]:
import create_vocab
import data_to_tensors
import model_implementation
from train import run_epoch, train

import torch
import torch.nn as nn
import torch.nn.functional as F

# Because their data looks like this: we have 3 main dictionaries:
* dictionary of all words in the sheets
* dictionary of all paths(they are already hashed and converted to digits)
* dictionary of targets(method names)


In [0]:
dict_path = '../data/java-small/java-small.dict.c2v'
word2idx, path2idx, target2idx, idx2target = create_vocab.create_vocab(dict_path)

* 1) data_iterator Returns 100 batches each(the size can be changed when calling the function)
* 2) from below you can see how the data is accessed, then 3 vectors are fed to the model(sorry for such names, I think you will understand from the arguments of the model that the output is)
* 3) well, at the output we have code_vector and the second vector-probability distribution between all labels in target_vocab(dictionary of method names)

In [0]:
path_for_train = '../data/java-small/java-small.train.c2v'
train_iterator = data_to_tensors.DatasetBuilder(path_for_train, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx, batch_size = 1024)

path_for_val = '../data/java-small/java-small.val.c2v'
val_iterator = data_to_tensors.DatasetBuilder(path_for_val, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx, batch_size = 1024)

# Train

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import random 

SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [0]:
model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
                             paths_vocab_size = len(path2idx), 
                             labels_num = len(target2idx))

In [0]:
N_EPOCHS = 10
LR = 1e-4

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss().to(DEVICE)

early_stop = False # ставите True и тогда будет обучение ток для одного батча
list_train_loss, list_val_loss, list_train_precision, list_val_precision,list_train_recall, list_val_recall, list_train_f1, list_val_f1 = train(model = model, optimizer = optimizer,
                                                                                                                                                criterion = criterion, train_loader = train_iterator,
                                                                                                                                                val_loader = val_iterator,
                                                                                                                                                epochs = N_EPOCHS, idx2target_vocab = idx2target, 
                                                                                                                                                scheduler=None, checkpoint=True, early_stop = early_stop)